## Vectara & Tonic Validate

In [1]:
import json
import logging
import os
import requests
# importing necessary functions from dotenv library
from dotenv import load_dotenv

# loading variables from .env file
load_dotenv()

True

In [2]:
TONIC_VALIDATE_API_KEY = os.getenv("TONIC_VALIDATE_API_KEY")
API_KEY = os.getenv("API_KEY")
CUSTOMER_ID = 43437896
CORPUS_ID = 3
SERVING_ENDPOINT = "api.vectara.io"

In [3]:
def _get_query_json(customer_id: int, corpus_id: int, query_value: str, num_results: int):
    """Returns a query JSON."""
    query = {
        "query": [
            {
                "query": query_value,
                "num_results": num_results,
                "corpus_key": [{"customer_id": customer_id, "corpus_id": corpus_id}],
            },
        ],
    }
    return json.dumps(query)


def get_query(query: str, num_results: int = 1):
    """Queries the data.
  
    Args:
        customer_id: Unique customer ID in vectara platform.
        corpus_id: ID of the corpus to which data needs to be indexed.
        query_address: Address of the querying server. e.g., api.vectara.io
        api_key: A valid API key with query access on the corpus.
  
    Returns:
        (response, True) in case of success and returns (error, False) in case of failure.
    """
    post_headers = {
        "customer-id": f"{CUSTOMER_ID}",
        "x-api-key": API_KEY
    }

    response = requests.post(
        f"https://{SERVING_ENDPOINT}/v1/query",
        data=_get_query_json(CUSTOMER_ID, CORPUS_ID, query, num_results),
        verify=True,
        headers=post_headers)

    if response.status_code != 200:
        logging.error("Query failed with code %d, reason %s, text %s",
                      response.status_code,
                      response.reason,
                      response.text)
        return response, False

    message = response.json()
    if (message["status"] and
            any(status["code"] != "OK" for status in message["status"])):
        logging.error("Query failed with status: %s", message["status"])
        return message["status"], False

    for response_set in message["responseSet"]:
        for status in response_set["status"]:
            if status["code"] != "OK":
                return status, False

    return message, True

In [4]:
SERVING_ENDPOINT

'api.vectara.io'

In [5]:
query = "How to get hire at gitlab?"

response, status = get_query(query)


In [6]:
response["responseSet"][0]["response"][0]["text"]

'In the effort to be as transparent as possible and avoid sending multiple competing GitLab offers, there will be an added step to the process after your final interview where you will discuss with your recruiter which role you’re interested in before any official offer documents can be sent.'

In [7]:
response["responseSet"][0]

{'response': [{'text': 'In the effort to be as transparent as possible and avoid sending multiple competing GitLab offers, there will be an added step to the process after your final interview where you will discuss with your recruiter which role you’re interested in before any official offer documents can be sent.',
   'score': 0.7387057,
   'metadata': [{'name': 'title', 'value': 'Interview Process FAQs'},
    {'name': 'lang', 'value': 'eng'},
    {'name': 'section', 'value': '1'},
    {'name': 'offset', 'value': '1664'},
    {'name': 'len', 'value': '292'}],
   'documentIndex': 0,
   'corpusKey': {'customerId': 0,
    'corpusId': 3,
    'semantics': 'DEFAULT',
    'dim': [],
    'metadataFilter': '',
    'lexicalInterpolationConfig': None},
   'resultOffset': 0,
   'resultLength': 292}],
 'status': [],
 'document': [{'id': 'Gitlab Interview Process FAQs.docx',
   'metadata': [{'name': 'X-TIKA:Parsed-By',
     'value': 'org.apache.tika.parser.microsoft.ooxml.OOXMLParser'},
    {'name

## Testing with benchmark QA

In [8]:
# Load questions from qa_pairs.json
import json
import tqdm
qa_pairs = []
with open('./data/QA_gitlab.json', 'r') as qa_file:
    qa_pairs = json.load(qa_file)

question_list = [qa_pair['question'] for qa_pair in qa_pairs]
print("Questions:\n" + "\n".join(question_list[:4]))


Questions:
How can candidates request a reasonable accommodation for their interview process at GitLab?
I've started the interview process at GitLab, but haven't heard back from anyone recently. What should I do?
Can I have the hiring team's email addresses so I can send them a note?
Can I interview for multiple roles at the same time?


In [9]:
answer_list = [qa_pair['answer'] for qa_pair in qa_pairs]
print("Answers:\n" + "\n".join(answer_list[:4]))

Answers:
Candidates can request a reasonable accommodation by reaching out to ces@gitlab.com. The Candidate Experience team along with Talent Acquisition leadership will collaborate with the candidate to define and accommodate their needs, which may include translation services, the use of additional services or assistive technologies, and assistance in setting up Closed Captioning for interviews.
Feel free to send an email to your Recruiter to get a status update on where you are in the interview process.
If you'd like to send a note to the hiring team, please send it to your Recruiter and they will forward it on.
Yes, you can apply for multiple roles at the same time. However, keep in mind that you will need to complete a full interview process for each role you apply to. Additionally, the recruiting team will only process 3 of your applications at a time based on which ones you decide to prioritize.


In [10]:
from tonic_validate import Benchmark
question_list = [qa_pair['question'] for qa_pair in qa_pairs]
answer_list = [qa_pair['answer'] for qa_pair in qa_pairs]

benchmark = Benchmark(questions=question_list, answers=answer_list)

In [11]:
from tonic_validate import ValidateScorer

scorer = ValidateScorer()
response_scores = scorer.score(benchmark, get_query, scoring_parallelism=2, callback_parallelism=2)

TypeError: tuple indices must be integers or slices, not str

In [12]:
from tonic_validate import ValidateScorer

# scorer = ValidateScorer(model_evaluator="gpt-3.5-turbo", max_parsing_retries=10)
scorer = ValidateScorer()
response_scores = scorer.score(benchmark, get_query)

TypeError: tuple indices must be integers or slices, not str

In [ ]:
import json
import os
from tonic_validate import ValidateApi
from tonic_validate.metrics import AnswerSimilarityMetric, RetrievalPrecisionMetric, AugmentationPrecisionMetric, AnswerConsistencyMetric
from llama_index.evaluation import TonicValidateEvaluator
import requests

In [ ]:
openai_responses, openai_context = [], []






# Go through all questions and get responses from openai assistant
for question in tqdm(question_list[len(openai_responses):]):
    # If there is an exception, try again until we reach 3 tries at max
    max_tries = 3
    while True:
        try:
            openai_response = get_response(question)
            openai_responses.append(openai_response)
	openai_context.append(openai_response[1])
            break
        except Exception as e:
            print(e)
            max_tries -= 1
            if max_tries == 0:
                raise e
            continue

In [ ]:
from tonic_validate import ValidateScorer, Benchmark, BenchmarkItem, LLMResponse, BenchmarkItem, Run
from tonic_validate.metrics import AnswerSimilarityMetric

score_calculator = RagScoresCalculator(
    model="gpt-4-1106-preview",
    answer_similarity_score=True,
)

# Score the results
openai_batch_scores = score_calculator.score_batch(
    question_list=question_list,
    reference_answer_list=answer_list,
    llm_answer_list=openai_responses,
)
openai_scores_df = openai_batch_scores.to_dataframe()
# Remove overall_score column since we are only using one stat
openai_scores_df = openai_scores_df.drop(columns=['overall_score'])
openai_scores_df.describe()